<a href="https://colab.research.google.com/github/bartek717/projectx/blob/main/projectxEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bert-score
!pip install nltk
!pip install pyrouge
!pip install sentencepiece
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=819a5bd10fdcd938df88ed4b702dad3dbeba303bf739def45b5c200403c8cae7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# evaluation of the models

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from bert_score import score
from pyrouge import Rouge155
from transformers import T5Tokenizer, T5Model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset
import torch
import pandas as pd
from google.colab import files
import time
import os
import numpy as np
import evaluate
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# import models

In [ ]:
model_output_dir_full = '/content/drive/My Drive/t5fine'

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")


# Load the fine-tuned model
model_full = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_full)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_full = model_full.to(device)

# model_cg_70 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_70)
# model_cg_50 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_50)
# model_cg_30 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_30)
# model_cg_10 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_10)
# model_cg_5 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_5)
# model_cg_3 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_3)
# model_cg_1 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_1)

# model_al_70 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_70)
# model_al_50 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_50)
# model_al_30 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_30)
# model_al_10 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_10)
# model_al_5 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_5)
# model_al_3 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_3)
# model_al_1 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_1)

# model_alcg_70 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_70)
# model_alcg_50 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_50)
# model_alcg_30 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_30)
# model_alcg_10 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_10)
# model_alcg_5 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_5)
# model_alcg_3 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_3)
# model_alcg_1 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_1)

In [ ]:

# generate summaries

In [ ]:
!git clone https://github.com/vgupta123/sumpubmed.git
%cd sumpubmed/

Cloning into 'sumpubmed'...
remote: Enumerating objects: 130808, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 130808 (delta 22), reused 17 (delta 6), pack-reused 130764
Receiving objects: 100% (130808/130808), 346.82 MiB | 24.88 MiB/s, done.
Resolving deltas: 100% (38368/38368), done.
Updating files: 100% (130765/130765), done.
/content/sumpubmed/sumpubmed/sumpubmed


In [ ]:
def process_data(text_dir, summary_dir):
    texts = []
    summaries = []
    for filename in os.listdir(text_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(text_dir, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())

            summary_file = "abst_"+(filename.split("_")[-1])
            with open(os.path.join(summary_dir, summary_file), 'r', encoding='utf-8') as file:
                summaries.append(file.read())

    return texts, summaries

texts_full, summaries_full = process_data('text/', 'shorter_abstract/')
texts = texts_full[int(0.999*len(texts_full)):]
reference_summaries = summaries_full[int(0.999*len(summaries_full)):]

In [ ]:
print(len(texts))

generation_parameters = {
    "max_new_tokens": 256,
    "num_beams": 5,
    "early_stopping": True,
    "no_repeat_ngram_size": 3,
    "temperature": 0.3
}

model_summaries = []
for text in texts:
    print('i')
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True).to(device)  # Move inputs to the same device
    summary_ids = model_full.generate(inputs, **generation_parameters)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    model_summaries.append(generated_summary)

33
i


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i


In [ ]:
# evaluate models

In [ ]:
print(model_summaries[1])
print(reference_summaries[1])

in this study, a semi-quantitative rt-pcr analysis was performed on glandular and glandular trichomes. amorpha- dig> -diene synthase and -linalool synthases were used to identify genes and markers for fast-track breeding in a. annua. to confirm the expression of some genes in glandular, we performed a global transcriptome characterization of the gene expression of terpene biosynthesis genes. the results indicate that the t-shapes of a tetracyclosaccharide and a sesquiterpene cyclass were expressed in the glandular or glandular cells. in addition, the cyp71av and dbr genes were found to be expressed in tyrosine phosphates, whereas the elongated cells of the sys were derived from adna and elisa. 
the medicinal plant artemisia annua is covered with filamentous trichomes and glandular, artemisinin producing trichomes.
therefore, metabolic activities of glandular and filamentous trichomes were investigated at the transcriptome level.
by laser pressure catapulting, glandular and filamentous 

In [ ]:
rouge_score = evaluate.load("rouge")

def compute_rouge_score(generated, reference):
    """
    Calculates and returns the ROUGE score using the evaluate library.
    """
    # We need to add '\n' to each line before sending it to ROUGE
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]

    # Calculate ROUGE scores
    rouge_scores = rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True
    )

    return rouge_scores


In [ ]:
rouge_scores = compute_rouge_score(model_summaries, reference_summaries)

# Access individual ROUGE scores (e.g., ROUGE-1, ROUGE-2, ROUGE-L)
print(rouge_scores)
rouge1_f = rouge_scores["rouge1"]
rouge2_f = rouge_scores["rouge2"]
rougeL_f = rouge_scores["rougeL"]

print(f"ROUGE-1 F1 Score: {rouge1_f:.4f}")
print(f"ROUGE-2 F1 Score: {rouge2_f:.4f}")
print(f"ROUGE-L F1 Score: {rougeL_f:.4f}")

{'rouge1': 0.3426875484505447, 'rouge2': 0.09880622035855771, 'rougeL': 0.17782382172025546, 'rougeLsum': 0.30955748060288435}
ROUGE-1 F1 Score: 0.3427
ROUGE-2 F1 Score: 0.0988
ROUGE-L F1 Score: 0.1778


In [ ]:
def bleu_mean(reference_summaries, model_summaries):
    """
    Calculates and returns the mean BLEU score.
    """
    bleu_scores = corpus_bleu(reference_summaries, model_summaries)
    return bleu_scores

In [ ]:
mean_bleu_score = bleu_mean(reference_summaries, model_summaries)
print(f"Mean BLEU Score: {mean_bleu_score:.4f}")

Mean BLEU Score: 0.0000


In [ ]:
def bert_score_mean(model_summaries, reference_summaries, lang="en"):
    """
    Calculates and returns the mean BertScore.
    """
    P, R, F1 = score(model_summaries, reference_summaries, lang=lang)
    mean_f1_score = F1.mean().item()

    return mean_f1_score

In [ ]:
mean_bert_score = bert_score_mean(model_summaries, reference_summaries, lang="en")
print(f"Mean BertScore: {mean_bert_score:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mean BertScore: 0.8369


In [ ]:
# do t-tests/anova for statistical significance